# Batch protein preparation

This notebook shows how to load a large amount of data and execute runs on them in parallel

In [ ]:
import os
import asyncio
from glob import glob
from datetime import datetime
from pathlib import Path

from pdbtools import (
    pdb_fetch,
    pdb_delhetatm,
    pdb_selchain,
    pdb_rplresname,
    pdb_keepcoord,
    pdb_selresname,
)

import rush

### 0) Setup

In [ ]:
# Set our token - ensure you have exported RUSH_TOKEN in your shell; or just
# replace the os.getenv with your token
TOKEN = os.getenv("RUSH_TOKEN")
URL = os.getenv("RUSH_URL")

In [ ]:
# Define our project information
DESCRIPTION = "rush-py batch notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "batch-prep"]
WORK_DIR = Path.home() / "qdx" / "rush-py-batch-prep"

# Set our inputs
PROTEIN_PDB_FOLDER_PATH = WORK_DIR / "proteins"

TARGET = "NIX_SSH"

In [ ]:
# |hide
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=True)

Ensure your workdir exists

In [ ]:
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(PROTEIN_PDB_FOLDER_PATH, exist_ok=True)

## Initialize our rush client and fetch available module paths

In [ ]:
# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(
    access_token=TOKEN, url=URL, workspace=WORK_DIR, batch_tags=TAGS
)

In [ ]:
# fetch datafiles
for pdb in ["1B39", "4QXI", "8FSU"]:
    complex = list(pdb_fetch.fetch_structure(pdb))
    protein = pdb_delhetatm.remove_hetatm(pdb_selchain.select_chain(complex, "A"))

    with open(PROTEIN_PDB_FOLDER_PATH / f"{pdb}_protein.pdb", "w") as f:
        for l in protein:
            f.write(str(l))

In [ ]:
help(client.prepare_protein)

Help on function prepare_protein in module rush.provider:

async prepare_protein(*args: [<class 'pathlib.Path'>], target: rush.graphql_client.enums.ModuleInstanceTarget | None = <ModuleInstanceTarget.NIX_SSH_2: 'NIX_SSH_2'>, resources: rush.graphql_client.input_types.ModuleInstanceResourcesInput | None = ModuleInstanceResourcesInput(gpus=1, gpu_mem=None, gpu_mem_units=None, cpus=None, nodes=None, mem=None, mem_units=None, storage=138, storage_units=<MemUnits.MB: 'MB'>, walltime=None, storage_mounts=None), tags: list[str] | None = None, restore: bool | None = None) -> [<class 'pathlib.Path'>, <class 'pathlib.Path'>]
    Prepare a PDB for downstream tasks: protonate, fill missing atoms, etc.
    
    Module version: github:talo/pdb2pqr/ff5abe87af13f31478ede490d37468a536621e9c#prepare_protein_tengu
    
    QDX Type Description:
    
        input_pdb: @bytes 
    
    ->
    
        output_qdxf: @[Conformer];
    
        output_pdb: @bytes
    
    
    
    :param input_pdb: An input 

## For each ligand, start a gmx + gmx_mmpbsa run

In [ ]:
proteins = map(lambda x: Path(x), glob(str(PROTEIN_PDB_FOLDER_PATH / "*.pdb")))

protein_outputs = []

for protein_path in proteins:
    print(protein_path)
    name = protein_path.stem
    (prepped_protein_qdxf, prepped_pdb) = await client.prepare_protein(
        protein_path,
        target=TARGET,
        tags=[name],
    )
    protein_outputs.append((name, prepped_protein_qdxf))

protein_outputs

/home/machineer/qdx/rush-py-batch-prep/proteins/1B39_protein.pdb
/home/machineer/qdx/rush-py-batch-prep/proteins/8FSU_protein.pdb
/home/machineer/qdx/rush-py-batch-prep/proteins/4QXI_protein.pdb


[('1B39_protein', Arg(id=243a4174-1aba-45c9-89e5-01da89e43fe0, value=None)),
 ('8FSU_protein', Arg(id=d7eacce8-f86d-4ebf-9d1e-e85c0cebfa5f, value=None)),
 ('4QXI_protein', Arg(id=9a5bdf8f-3ab9-485e-9efb-6999f99ea47e, value=None))]

## Report progress
This will show the status of all of your runs

In [ ]:
status = await client.status(group_by="path")
print(f"{'Module':<20} | {'Status':<20} | Count")
print("-" * 50)
for module, (status, path, count) in status.items():
    print(f"{path:<20} | {status:<20} | {count}")

Module               | Status               | Count
--------------------------------------------------
prepare_protein      | ModuleInstanceStatus.RESOLVING | 3


## Download Results
This will retrieve results for your completed module_instances

In [ ]:
await asyncio.gather(
    *[
        output[1].download(filename=f"protein_{output[0]}.qdxf.json")
        for output in protein_outputs
    ],
)

2024-01-24 10:34:42,469 - rush - INFO - Argument 243a4174-1aba-45c9-89e5-01da89e43fe0 is now ModuleInstanceStatus.RESOLVING
2024-01-24 10:34:42,591 - rush - INFO - Argument 9a5bdf8f-3ab9-485e-9efb-6999f99ea47e is now ModuleInstanceStatus.RESOLVING
2024-01-24 10:34:42,635 - rush - INFO - Argument d7eacce8-f86d-4ebf-9d1e-e85c0cebfa5f is now ModuleInstanceStatus.RESOLVING
2024-01-24 10:34:46,943 - rush - INFO - Argument 243a4174-1aba-45c9-89e5-01da89e43fe0 is now ModuleInstanceStatus.ADMITTED
2024-01-24 10:34:46,992 - rush - INFO - Argument 9a5bdf8f-3ab9-485e-9efb-6999f99ea47e is now ModuleInstanceStatus.ADMITTED
2024-01-24 10:34:47,054 - rush - INFO - Argument d7eacce8-f86d-4ebf-9d1e-e85c0cebfa5f is now ModuleInstanceStatus.ADMITTED
2024-01-24 10:35:00,097 - rush - INFO - Argument 243a4174-1aba-45c9-89e5-01da89e43fe0 is now ModuleInstanceStatus.DISPATCHED
2024-01-24 10:35:05,571 - rush - INFO - Argument 243a4174-1aba-45c9-89e5-01da89e43fe0 is now ModuleInstanceStatus.RUNNING
2024-01-24 1

[None, None, None]

## Check failures
This will retrieve failed runs in your workspace history

In [ ]:
for instance_id, (status, name, count) in (await client.status()).items():
    if status.value == "FAILED":
        async for log_page in client.logs(instance_id, "stderr"):
            for log in log_page:
                print(log)